In [1]:
!pip install transformers sentencepiece


In [2]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Choose model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
# TESTING
job_ad = "we are looking for a senior engineer"
# Format the prompt
prompt = f"Classify the seniority (junior, senior, etc.) of the job: {job_ad}"

# Tokenize
input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True).to(device) # Move input_ids to the same device as the model

# Generate output
job_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

# Decode and print
print("Summary:", tokenizer.decode(job_ids[0], skip_special_tokens=True))

Summary: job: we are looking for senior engineer.


In [5]:
import os
from google.colab import drive

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/AB_job_data_files'

import os
from google.colab import drive

# find location of different files:
file_location = {}
# Check if the folder exists
if os.path.exists(folder_path):
  os.chdir(folder_path)
  for num, f in enumerate(os.listdir()):
    file_location[f] = os.path.join(folder_path, f)

else:
  print(f"Folder not found: {folder_path}")

print(file_location)

Mounted at /content/drive
{'seniority_labelled_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/seniority_labelled_test_set.csv', 'unlabelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/unlabelled_development_set.csv', 'salary_labelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/salary_labelled_development_set.csv', 'seniority_labelled_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/seniority_labelled_development_set.csv', 'work_arrangements_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/work_arrangements_test_set.csv', 'salary_labelled_test_set.csv': '/content/drive/MyDrive/AB_job_data_files/salary_labelled_test_set.csv', 'work_arrangements_development_set.csv': '/content/drive/MyDrive/AB_job_data_files/work_arrangements_development_set.csv', 'results': '/content/drive/MyDrive/AB_job_data_files/results', 'wandb': '/content/drive/MyDrive/AB_job_data_files/wandb', 't5-small-work-arrangement-finetuned': '/content/d

In [6]:
def seniority_estimate(job_ad):

    # Format the prompt
    prompt = f"Identify the seniority (e.g. senior, junior etc.) from the job ad: {job_ad}"

    # Tokenize
    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True).to(device) # Move input_ids to the same device as the model

    # Generate output
    job_ids = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)

    # Decode and return
    return tokenizer.decode(job_ids[0], skip_special_tokens=True)

In [7]:
# DEV set
import pandas as pd

senior_dev = pd.read_csv(file_location['seniority_labelled_development_set.csv'])

results = 0
count = 0
for index, row in senior_dev.iterrows():
    predicted_label = seniority_estimate(row['job_ad_details'])
    true_label =  row['y_true']

    print(predicted_label, '----', true_label)
    print(predicted_label == true_label)
    if predicted_label == true_label:
        results += 1
    count += 1

print("count:", count)
print("result:", results)
print("correctly indentified:", results/count)


/strong>br> strong>strong>About the company/strong>br> &nbsp;&nbsp;&nbsp;br> strong>strong>About the company/strong>br> br> strong>About the company/strong>br> br> strong ---- intermediate
False
/p>/p>ul>li>A true global leader in the lifestyle and entertainment industries, is seeking to hire an experienced and passionate Marketing Manager to join an established and high performing team in Sydney's CBD. You will own and execute mid to long term national and global marketing plans, set the digital strategy with view to reach and acquire the largest possible target audience/li>li>Coach, lead and develop the team ---- senior
False
/p> /li>li> p>Accreditation and regulatory compliance, including developing best practice initiatives through internal auditing and action plan development and monitoring/p> /li>li> p>Be involved in the review and development of organisational policies and procedures/p> /li>li> p>Lead and promote innovation in care ---- lead
False
/p>&nbsp;/p> p>This new positio

KeyboardInterrupt: 

In [8]:
# TEST set
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

seniority_test = pd.read_csv(file_location['seniority_labelled_test_set.csv'])

results = 0
count = 0

# Get labels and convert to lowercase for consistency
true_labels = [label.lower() for label in seniority_test['y_true']]

# Make predictions
predictions = []

for index, row in seniority_test.iterrows():
    predicted_label = seniority_estimate(row['job_ad_details'])

    true_label =  row['y_true']

    #print(work_arrang(row['job_ad']), '----', row['y_true'])
    #print(predicted_label == true_label)
    if predicted_label == true_label:
        results += 1
    count += 1
    predictions.append(predicted_label)

print("count:", count)
print("result:", results)
print("correctly indentified:", results/count)




# calculate metrics
accuracy = accuracy_score(true_labels, predictions)
report = classification_report(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

# print results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(report)
print("\nConfusion Matrix:")
print(conf_matrix)



count: 689
result: 0
correctly indentified: 0.0
Accuracy: 0.0000

Classification Report:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          precision    recall  f1-score   support

                                                                                                                                                                                         &amp; development programs are highly recognised as the best on offer within the retail industry./strong>/li> li>Exceptional potential for career growth./strong>/p

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_